In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import csv
import random
import json
import os
import re
import openai
from dotenv import load_dotenv
import time
import sys
from IPython.display import display, Markdown, Latex
import pandas as pd
import sympy as sp
#from latex2sympy2 import latex2sympy, latex2latex 
from sympy import simplify, cos, sin, expand, Eq
import eval_functions
import openai_model
from sympy import Mul, symbols
from sympy import symbols, lambdify
from sympy.parsing.latex import parse_latex
from math import pi

In [ ]:
# TODO: make file and load in function 
evalsum_df = pd.DataFrame()
evalsum_df['problem_type'] = np.array(['overall','nondimensionalization','root_finding','ode','integral'])
evalsum_df['problem_num'] = np.array([437,200,63,54,120])
evalsum_df['gpt4turbo_0shot'] = np.array([13.4, 6.04, 32.1, 15.7, 14.9])
evalsum_df['gpt4turbo_1shot'] = np.array([37.6, 36.5, 50.4, 20.3, 40.5])
evalsum_df['gpt4turbo_5shot'] = np.array([46.3, 48.6, 58.1, 34.7, 41.4])
evalsum_df['gpt3.5turbo_0shot'] = np.array([4.54, 5.05, 17.2, 7.41, 3.33])
evalsum_df['gpt3.5turbo_1shot'] = np.array([14.8, 6.11, 28.9, 22.6, 18.2])
evalsum_df['gpt3.5turbo_5shot'] = np.array([25.6, 24.3, 29.3, 32.1, 23.1])
evalsum_df['llama3_0shot'] = np.array([3.98, 0.50, 11.3, 11.6, 2.52])
evalsum_df['llama3_5shot'] = np.array([21.1, 17.9, 17.5, 21.9, 28.1])
evalsum_df['codellama_0shot'] = np.array([3.13, 0.00, 8.0, 14.1, 0.50])
evalsum_df['codellama_5shot'] = np.array([10.3, 8.41, 13.1, 15.3, 9.57])

In [ ]:
# by model
fig, ax  = plt.subplots(figsize=(5,5))
ax.plot(np.arange(3),evalsum_df.iloc[0,2:5],color = 'tab:blue',marker='o',label ='gpt-4')
for i in range(1,5):
    ax.plot(np.arange(3),evalsum_df.iloc[i,2:5],'--',color = 'tab:blue',marker='o',alpha=0.3)
ax.plot(np.arange(3),evalsum_df.iloc[0,5:8],color = 'tab:purple',marker='o',label ='gpt-3.5')
for i in range(1,5):
    ax.plot(np.arange(3),evalsum_df.iloc[i,5:8],'--',color = 'tab:purple',marker='o',alpha=0.3)
ax.plot([0,2],evalsum_df.iloc[0,8:10],color = 'tab:orange',marker='o',label ='llama3-7b')
for i in range(1,5):
    ax.plot([0,2],evalsum_df.iloc[i,8:10],'--',color = 'tab:orange',marker='o',alpha=0.3)
ax.plot([0,2],evalsum_df.iloc[0,10:12],color = 'tab:red',marker='o',label ='codellama-13b')
for i in range(1,5):
    ax.plot([0,2],evalsum_df.iloc[i,10:12],'--',color = 'tab:red',marker='o',alpha=0.3)
ax.set_xticks(np.arange(3))
ax.set_xticklabels(['0-shot','1-shot','5-shot'])
ax.set_ylabel('Accuracy (%)')
#ax.legend()
ax.legend(loc='upper center', bbox_to_anchor=(0.5, -0.05),
          fancybox=True, shadow=True, ncol=5)